## Import Libraries

In [3]:
import importlib
import os

try:
    importlib.reload(dataloaders)
    importlib.reload(models)
except:
    import dataloaders
    import models

## Load Data

In [5]:
file = os.path.abspath("all_molecules.csv")
dataloader = dataloaders.DataLoader(file)

In [6]:
descriptors = dataloader.get_molecular_descriptors()
descriptors.shape

(2000, 209)

In [7]:
labels = dataloader.get_labels()
labels.shape

(2000, 1)

## PCA Analysis

## Model training

In [8]:
model = models.Model(descriptors, labels)

descriptors = model.preprocessing()

(2000, 209)


c:\Users\marti\OneDrive - TU Eindhoven\Documenten\Master\Q4\Advanced Programming\8CC00Code\GroupAssignment\models.py:12: RuntimeWarning: invalid value encountered in divide
  self.x = (self.x - min_val) / (max_val - min_val)


In [9]:
model.train()

array([[8.61052080e-01, 8.61052080e-01, 6.27928938e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.88704936e-01, 7.88704936e-01, 3.62008958e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.88571281e-01, 6.88571281e-01, 9.12075617e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [7.15574023e-01, 7.15574023e-01, 2.18068619e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.39471775e-01, 8.39471775e-01, 8.23013709e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.52619025e-01, 8.52619025e-01, 3.40075790e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
accuracy = model.test()
print(accuracy)